# Yahoo Fantasy Football Weekly Data Pulls
## Pulls for a weekly report that goes out to league members

### Set up

In [ ]:
import pandas as pd
import numpy as np
import logging
import os
import pprint
import warnings
import json
import csv
from datetime import datetime

import pytest
from dotenv import load_dotenv

from yfpy import Data
from yfpy.models import Game, StatCategories, User, Scoreboard, Settings, Standings, League, Player, Team, \
    TeamPoints, TeamStandings, Roster
from yfpy.query import YahooFantasySportsQuery

In [ ]:
# Settings from yfpy package
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)
warnings.simplefilter("ignore", ResourceWarning)
env_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('private.json'))), ".env")
load_dotenv(dotenv_path=env_path)
print_output = True 
browser_callback = True
auth_dir = "."
data_dir = os.path.join(os.path.dirname(os.path.abspath('private.json')), "test_output")

current_week = 9
season = '2021'
game_code = "nfl"
if season == '2019':
    game_key = "390"
    league_id = "1021512"
elif season == '2020':
    game_key = "399"
    league_id = "234883"
elif season == '2021':
    game_key = '406'
    league_id = '435810'

with open(r'C:/Users/bradr/Python/fantasy_football/token.json', 'r') as json_file:
    private = json.load(json_file)

# Instantiate yfpy objects
yahoo_data = Data(data_dir)
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id,
    game_id=game_key,
    game_code=game_code,
    offline=False,
    all_output_as_json=False,
    consumer_key=private['consumer_key'],
    consumer_secret=private['consumer_secret'],
    browser_callback=browser_callback
)

### Get the metrics in the current Yahoo home page like record, points, transaction amounts

In [ ]:
standings_data = yahoo_query.get_league_standings()

In [ ]:
# Convert Standings Object to json
parsed_json = (json.loads(str(standings_data)))

# Initialize a list that can store the data
team_list = []
# Iterate over the Standings json to get teamids and their respective points
for team in parsed_json['teams']:
    team_list.append((team['team']['name'],
                      str(team['team']['team_standings']['outcome_totals']['wins']),
                      str(team['team']['team_standings']['outcome_totals']['losses']),
                      str(team['team']['team_standings']['outcome_totals']['ties']),
                      team['team']['team_standings']['outcome_totals']['percentage'],
                      team['team']['waiver_priority'],
                      team['team']['team_standings']['points_for'],
                      team['team']['team_standings']['points_against'],
                      team['team']['team_standings']['streak']['type'].upper()[0] + team['team']['team_standings']['streak']['value'],
                      team['team']['number_of_moves'],
                      team['team']['number_of_trades'],
                      team['team']['draft_grade'],
                      team['team']['team_logos']['team_logo']['url'],
                      team['team']['team_key'],
                      int(team['team']['team_id'])))

# Sort the team list by the team id
team_list = sorted(team_list)
teams = pd.DataFrame(team_list, columns = ['team_name','wins','losses','ties','win_pct','waiver_priority','points_for','points_against',
                                           'streak','moves','trades','draft_grade','team_logo','team_key','team_id'])\
.sort_values('win_pct', ascending = False)
teams.to_csv(f'output/{season}_current_scoreboard.csv')

### Get the weekly projected points to see how accurate they are and if people perform above or below expectation

In [ ]:
projected_points_list = []
for week in range(1, current_week + 1):
    scoreboard = yahoo_query.get_league_scoreboard_by_week(chosen_week = week)
    for game in range(0, len(scoreboard.matchups)):
        for team in range(0,2):
            projected_points_list.append((
                scoreboard.matchups[game]['matchup'].teams[team]['team'].team_id,
                scoreboard.matchups[0]['matchup'].week,
                scoreboard.matchups[game]['matchup'].teams[team]['team'].team_points.total,
                scoreboard.matchups[game]['matchup'].teams[team]['team'].team_projected_points.total,
                scoreboard.matchups[game]['matchup'].winner_team_key
            ))
proj_points_df = pd.DataFrame(projected_points_list, columns = ['team_id','week','total_points','projected_points','winner_team_key'])\
.sort_values('week', ascending = True)
proj_points_df.to_csv(f'output/{season}_week{current_week}_projected_points.csv')
# proj_points_df

### Get all teams rosters and starters for each week of season. To be used to calculate lineup efficiency

In [ ]:
roster_list = []
for week in range(1,17):
    for league_team in range(1,11): 
        roster_week = yahoo_query.get_team_roster_player_info_by_week(team_id = league_team, chosen_week = week)
        for player in range(0,len(roster_week)):
            roster_list.append((league_team,
                                week,
                                roster_week[player]['player'].display_position, 
                                roster_week[player]['player'].selected_position.position,
                                roster_week[player]['player'].full_name,
                                roster_week[player]['player'].player_key.split(".")[2],
                                roster_week[player]['player'].player_points_value,
                                roster_week[player]['player'].headshot_url))
    
roster_week_df = pd.DataFrame(roster_list, columns = ['team_id',
                                                      'week',
                                                      'position',
                                                      'lineup_position',
                                                      'full_name',
                                                      'player_key',
                                                      'fantasy_points',
                                                      'player_pic'])
roster_week_df.to_csv(f'output/{season}_roster_week.csv', index = False)
# roster_week_df

### Get the season transactions to date. To be used for our keeper rules to see if a player was dropped at all through season.

In [ ]:
transactions_list = []
transactions = yahoo_query.get_league_transactions()

num_player_list = []
for move in range(0,len(transactions)):
    num_player_list.append(len(transactions[move]['transaction'].players))
max_num_player = max(num_player_list)

transaction_comp_cols = ['transaction_id',
                         'timestamp',
                         'type',
                         'status']

for move in range(0,len(transactions)):
    transaction_components = (transactions[move]['transaction'].transaction_id,
                          datetime.utcfromtimestamp(int(transactions[move]['transaction'].timestamp)).strftime('%Y-%m-%d %H:%M:%S'),
                          transactions[move]['transaction'].type,
                          transactions[move]['transaction'].status)
    if len(transactions[move]['transaction'].players) == 1:
        transactions_list.append(transaction_components+(
                                  transactions[move]['transaction'].players['player'].transaction_data.source_type,
                                  transactions[move]['transaction'].players['player'].transaction_data.destination_team_key,
                                  transactions[move]['transaction'].players['player'].transaction_data.destination_team_name,
                                  transactions[move]['transaction'].players['player'].player_id,
                                  transactions[move]['transaction'].players['player'].name.full
                                  )) 

    for num in range(2,max_num_player+1):
        
        if len(transactions[move]['transaction'].players) == num:
            player_cols = ()
            player_col_names = []
            for i in range(0,len(transactions[move]['transaction'].players)):
                player_values = (transactions[move]['transaction'].players[i]['player'].transaction_data.source_type,
                                 transactions[move]['transaction'].players[i]['player'].transaction_data.destination_team_key,
                                 transactions[move]['transaction'].players[i]['player'].transaction_data.destination_team_name,
                                 transactions[move]['transaction'].players[i]['player'].player_id,
                                 transactions[move]['transaction'].players[i]['player'].name.full)
                player_cols = player_cols + player_values
                j = i + 1
                player_col_names = player_col_names + [f'source_type_{j}',
                                                       f'destination_team_key_{j}',
                                                       f'destination_team_name_{j}',
                                                       f'player_id_{j}',
                                                       f'player_name_{j}']
                
                if num == max_num_player:
                    final_player_col_names = player_col_names

            transactions_list.append(transaction_components+player_cols)  
        

transactions_df = pd.DataFrame(transactions_list, columns = transaction_comp_cols+list(final_player_col_names))
transactions_df.to_csv(f'output/{season}_transactions.csv', index = False)
# transactions_df

### Get team info to stay up to date with team names and logos if they change during the season

In [ ]:
standings_data = yahoo_query.get_league_standings()
	
# Convert Standings Object to json
parsed_json = (json.loads(str(standings_data)))

# Initialize a list that can store the data
team_list = []
# Iterate over the Standings json to get teamids and their respective points
for team in parsed_json['teams']:
    team_list.append((int(team['team']['team_id']),
                      team['team']['name'],
                      team['team']['team_key'],
                      team['team']['draft_grade'],
                      team['team']['team_logos']['team_logo']['url']))

# Sort the team list by the team id
team_list = sorted(team_list)
teams = pd.DataFrame(team_list, columns = ['team_id','team_name','team_key','draft_grade','team_logo'])
teams.to_csv(f'output/{season}_teams.csv', index = False)
# teams